In [ ]:
# PyTorch Image Classifier - Cleaned Version
# This code implements a PyTorch-based image classification model using a pre-trained ResNet-18 architecture, fine-tuned on a custom dataset. Here's a summary of its main steps:
# Imports: Essential PyTorch, torchvision, and data handling libraries are imported for model building, training, and data management.

# Data Transforms: The get_data_transforms function defines preprocessing steps for input images, such as resizing, converting to tensors, and normalizing with ImageNet statistics.

# Data Loading: The get_dataloaders function loads training and validation datasets from a specified directory using ImageFolder, which assumes images are organized in subdirectories for each class. It also creates data loaders for batch processing.

# Model Setup: The initialize_model function sets up a ResNet-18 model pre-trained on ImageNet. It replaces the final fully connected layer to match the number of classes in the dataset, and freezes earlier layers for feature extraction.

# Training: The train_model function trains the model over multiple epochs, tracking loss and accuracy for both the training and validation phases. It uses cross-entropy loss and an SGD optimizer, and updates weights only during the training phase.

# Main Execution: In the main block, the code sets paths and hyperparameters (batch size, number of epochs). It then loads the dataset, initializes the model, and trains it, saving the best model weights based on validation accuracy to a file called best_model.pth.

# This structure allows for efficient training and evaluation of an image classifier with transfer learning, leveraging a pre-trained model while customizing it for a new dataset.

In [3]:
# --- Imports ---
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 23.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Matplotlib is building the font cache; this may take a moment.


In [4]:
# --- Data Transforms ---
def get_data_transforms(input_size=224):
    return transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [5]:
# --- Data Loading ---
def get_dataloaders(data_dir, batch_size=32, num_workers=4):
    transform = get_data_transforms()
    train_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform)
    val_ds = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform)
    
    dataloaders = {
        'train': DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers),
        'val': DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    }
    dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}
    class_names = train_ds.classes

    return dataloaders, dataset_sizes, class_names

In [6]:
# --- Training Function ---
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}\n' + '-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [8]:
# --- Main Execution ---
if __name__ == '__main__':
    data_dir = '/Users/eabowman/Dropbox/LichenProject/dataset'
    batch_size = 32
    num_epochs = 25
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    dataloaders, dataset_sizes, class_names = get_dataloaders(data_dir, batch_size)
    model = initialize_model(num_classes=len(class_names))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

    model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs, device)

    torch.save(model.state_dict(), 'best_model.pth')

FileNotFoundError: Couldn't find any class folder in /Users/eabowman/Dropbox/LichenProject/dataset/train.